In [10]:
from sympy import Ynm, Symbol, diff, simplify, sqrt, conjugate
from sympy import latex
from sympy.vector import CoordSys3D, gradient
from sympy.abc import l, m
import IPython.display as disp

# Helpers

In [11]:
def pprint(x, simplify_expression: bool = False):
    if simplify_expression:
        return disp.display(simplify(x.expand(func=True)))
    return disp.display(x)


def to_latex(x):
    print(
        latex(x)
        .replace("\\mathbf{{phi}_{S}}", "\\phi")
        .replace("\\mathbf{{theta}_{S}}", "\\theta")
        .replace("\\mathbf{{r}_{S}}", "r")
    )


# Coordinate System

In [12]:
S = CoordSys3D(
    "S",
    transformation="spherical",
    variable_names=("r", "theta", "phi"),
    vector_names=("e_r", "e_t", "e_p"),
)

def conjugate_vector(x):
    res = conjugate(x.dot(S.e_r)) * S.e_r + conjugate(x.dot(S.e_t)) * S.e_t + conjugate(x.dot(S.e_p)) * S.e_p
    return res.subs({conjugate(S.theta): S.theta, conjugate(S.phi): S.phi, conjugate(S.r): S.r,
                     conjugate(m): m, conjugate(l): l})

# Complex Vector Spherical Harmonics

In [13]:
# These are complex for now.
def Y(l, m):
    return Ynm(l, m, S.theta, S.phi) * S.e_r


def Psi(l, m):
    grad_Y = gradient(Ynm(l, m, S.theta, S.phi))
    return S.r * grad_Y


def Phi(l, m):
    grad_Y = gradient(Ynm(l, m, S.theta, S.phi))
    return (S.r * S.e_r).cross(grad_Y)

In [14]:
pprint(Y(l, m))
pprint(Psi(l, m))
pprint(Phi(l, m))

(Ynm(l, m, S.theta, S.phi))*S.e_r

(m*cot(S.theta)*Ynm(l, m, S.theta, S.phi) + sqrt((l - m)*(l + m + 1))*exp(-S.phi*I)*Ynm(l, m + 1, S.theta, S.phi))*S.e_t + (I*m*Ynm(l, m, S.theta, S.phi)/sin(S.theta))*S.e_p

(-I*m*Ynm(l, m, S.theta, S.phi)/sin(S.theta))*S.e_t + (m*cot(S.theta)*Ynm(l, m, S.theta, S.phi) + sqrt((l - m)*(l + m + 1))*exp(-S.phi*I)*Ynm(l, m + 1, S.theta, S.phi))*S.e_p

In [15]:
pprint(conjugate_vector(Y(l, m)))
pprint(conjugate_vector(Psi(l, m)))
pprint(conjugate_vector(Phi(l, m)))

((-1)**(2*m)*exp(-2*S.phi*I*m)*Ynm(l, m, S.theta, S.phi))*S.e_r

((-1)**(2*m)*m*exp(-2*S.phi*I*m)*cot(S.theta)*Ynm(l, m, S.theta, S.phi) + (-1)**(2*m + 2)*exp(S.phi*I)*exp(-2*S.phi*I*(m + 1))*conjugate(sqrt((l - m)*(l + m + 1)))*Ynm(l, m + 1, S.theta, S.phi))*S.e_t + (-(-1)**(2*m)*I*m*exp(-2*S.phi*I*m)*Ynm(l, m, S.theta, S.phi)/sin(S.theta))*S.e_p

((-1)**(2*m)*I*m*exp(-2*S.phi*I*m)*Ynm(l, m, S.theta, S.phi)/sin(S.theta))*S.e_t + ((-1)**(2*m)*m*exp(-2*S.phi*I*m)*cot(S.theta)*Ynm(l, m, S.theta, S.phi) + (-1)**(2*m + 2)*exp(S.phi*I)*exp(-2*S.phi*I*(m + 1))*conjugate(sqrt((l - m)*(l + m + 1)))*Ynm(l, m + 1, S.theta, S.phi))*S.e_p

In [16]:
to_latex(Y(l, m).cross(Psi(l, m)).dot(Phi(l, m)))

- \frac{m^{2} \left(Y_{l}^{m}\left(\theta,\phi\right)\right)^{3}}{\sin^{2}{\left(\theta \right)}} + \left(m \cot{\left(\theta \right)} Y_{l}^{m}\left(\theta,\phi\right) + \sqrt{\left(l - m\right) \left(l + m + 1\right)} e^{- \phi i} Y_{l}^{m + 1}\left(\theta,\phi\right)\right)^{2} Y_{l}^{m}\left(\theta,\phi\right)


# Integration

In [17]:
from sympy import pi, sin, cos
from sympy import conjugate
from sympy.vector import ParametricRegion, vector_integrate
from sympy.abc import theta, phi

In [40]:
def integrate_on_sphere(func):
    print("Integrand:")
    pprint(func, simplify_expression=True)
    
    sphere = ParametricRegion(
        (sin(theta) * cos(phi), sin(theta) * sin(phi), cos(theta)),
        (theta, 0, pi),
        (phi, 0, 2 * pi),
    )
    print("Integral over sphere:")
    expr = simplify(func.expand(func=True)).subs({S.theta: theta, S.phi: phi, S.r: 1})
    return vector_integrate(expr, sphere)


pprint(
    integrate_on_sphere((Y(1, 1).cross(Phi(1, 1))).dot(conjugate_vector(Psi(1, 1)))),
    simplify_expression=True,
)

Integrand:


3*sqrt(6)*(cos(S.theta)**2 + 1)*exp(S.phi*I)*sin(S.theta)/(32*pi**(3/2))

Integral over sphere:


0